# requirements

## Pinecone

Create an index `langchain-demo` with a dimension of `1536`, if using OpenAI.

See https://platform.openai.com/docs/guides/embeddings/second-generation-models 

# imports

In [10]:
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import MarkdownTextSplitter
from langchain.vectorstores import Pinecone
import os
import pinecone


# vectorization

In [11]:
load_dotenv()

pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"])

embeddings = OpenAIEmbeddings()

index_name = "langchain-demo"

text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=100)
# traverse the ./content directory recursively and load each file that contains plain text

docs = []
for root, _, files in os.walk("./content"):
    for file in files:
        if file.endswith(".txt") or file.endswith(".md"):
            filepath = os.path.join(root, file)
            print('.', end='')
            with open(filepath, "r") as f:
                text = f.read()
                metadata={'source': filepath}
                doc = Document(page_content=text, metadata=metadata)
                split_docs = text_splitter.split_documents([doc])
                docs.extend(split_docs)

docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)


.....................

# QA bot

In [12]:
from IPython.display import display
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import ipywidgets as widgets

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True
)

def demo(query):
    result = qa(
        {
            "query": query['new']
        }
    )

    if result["result"]:
        print(result["result"], end=' ')
        print(result["source_documents"][0].metadata)
    else:
        print("I'm sorry I don't have any idea about this ask. Try a different question?")


# Create text widget for input
input_text = widgets.Text(continuous_update=False, layout=widgets.Layout(width='100%'), placeholder='What do you want to know?')

# Display widget
display(input_text)

input_text.observe(demo, names='value')

Text(value='', continuous_update=False, layout=Layout(width='100%'), placeholder='What do you want to know?')

 I don't know. {'source': './content/home.md'}
 You can find information about setting up the CasaOS development environment on the Contribute page. {'source': './content/contribute/development.md'}
